#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Checking the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

from cassandra.cluster import Cluster
cluster = Cluster()

# A session To establish connection and begin executing queries
session = cluster.connect()

#### Creating Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Query 1: Find the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### - Creating session_songs table
#### - Partition Key is a composite key of (session_id, item_in_session) since we are retrieving a song in a certain session.
#### - Partition Key has to be composite here to prevent records from being overwritten if only session_id or item_in_session is chosen as partition key.
#### - No clustering columns.

In [8]:
## Creating Table session_songs
query = "CREATE TABLE IF NOT EXISTS session_songs "
query = query + "(session_id int, item_in_session int , artist TEXT, song_title TEXT, length DOUBLE, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
## Reading the events from datafile
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Inserting records into session_songs table
        query = "INSERT INTO session_songs (session_id, item_in_session, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute( query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])) )

In [10]:
## Executing the SELECT query to find the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "SELECT artist, song_title, length FROM session_songs WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, ", ", row.song_title, ", ", row.length)

Faithless ,  Music Matters (Mark Knight Dub) ,  495.3073


### Query 2: Find only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### - Creating user_session_songs table
#### - Partition Key is a composite key of (user_id, session_id) since we are retrieving the songs for a certain user in a certain session. 
#### - Partition Key has to be composite here to prevent records from being overwritten if only user_id or session_id is chosen as partition key.
#### - Clustering column is item_in_session as we want the results to be sorted by itemInSession.

In [11]:
query = "CREATE TABLE IF NOT EXISTS user_session_songs "
query = query + "(user_id int, session_id int, item_in_session int, first_name TEXT, last_name TEXT, artist TEXT, song_title TEXT, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
## Reading the events from datafile
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Inserting records into user_session_songs table
        query = "INSERT INTO user_session_songs (user_id, session_id, item_in_session, first_name, last_name, artist, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute( query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]) )

In [13]:
## Executing the SELECT query to find only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT artist, song_title, first_name, last_name FROM user_session_songs WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, ", ", row.song_title, ", ", row.first_name, row.last_name)   

Down To The Bone ,  Keep On Keepin' On ,  Sylvie Cruz
Three Drives ,  Greece 2000 ,  Sylvie Cruz
Sebastien Tellier ,  Kilometer ,  Sylvie Cruz
Lonnie Gordon ,  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) ,  Sylvie Cruz


### Query 3: Find every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### - Creating user_songs table
#### - Partition Key is a composite key of (song_title, user_id) since we are retrieving all the users who have listened for a specific song.
#### - Partition Key has to be composite here to prevent records from being overwritten if only song_title or user_id is chosen as partition key.
#### - No Clustering columns.
#### - Note that here overwriting will happen if the same user listens to this song in the same session more than once or in different sessions, but this is fine as we want the name of the user which is the same in all song playings. 

In [14]:
## Creating Table user_songs
query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + "(song_title TEXT, user_id INT, first_name TEXT, last_name TEXT, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
## Reading the events from datafile
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Inserting records into user_songs table
        query = "INSERT INTO user_songs (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute( query, (line[9], int(line[10]), line[1], line[4]) )

In [16]:
## Executing the SELECT query to find every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT first_name, last_name FROM user_songs WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)     

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [17]:
## Dropping the tables before closing out the sessions

query = "DROP TABLE IF EXISTS session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
query = "DROP TABLE IF EXISTS user_session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


query = "DROP TABLE IF EXISTS user_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [18]:
## Closing session
session.shutdown()
cluster.shutdown()